In [14]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

In [6]:
data = pd.read_csv('data-logistic.csv', header = None)
y = data.iloc[:,0]
X = data.iloc[:,1:]
X1, X2 = X[1], X[2]

In [7]:
#получение след шага градиентного спуска
def NextStep(w, C = 0, k = 0.1):
    w1, w2 = w[0], w[1]
    l = len(y)
    w1_next = w1+k/l*np.sum([Y[i]*X1[i]*(1-1/(1+np.exp(-Y[i]*(w1*X1[i]+w2*X2[i])))) for i in range(l)])-k*C*w1
    w2_next = w2+k/l*np.sum([Y[i]*X2[i]*(1-1/(1+np.exp(-Y[i]*(w1*X1[i]+w2*X2[i])))) for i in range(l)])-k*C*w2
    return [w1_next, w2_next]

In [8]:
from scipy.spatial.distance import euclidean
#подбор таких w1, w2, чтобы они отличались от значений прошлого шага менее чем на eps
def GetW1W2(C = 0):
    w = [0, 0]
    eps = 1e-5
    k = 0
    while True:
        w_next = NextStep(w, C)
        if (euclidean(w_next, w) < eps):
            break
        w = w_next
        k += 1
    return w, k
    

In [9]:
#просто вызов функций, что выше без регуляризации и с С=10
w_0, iter1 = GetW1W2()
w_10, iter2 = GetW1W2(10)
print(w_0, iter1)
print(w_10, iter2)

[0.28780442348832036, 0.09198995719421853] 243
[0.028561965517012657, 0.024783655436404754] 7


In [10]:
#оценим по метрике AUC-ROC, тк тут речь о вероятности принадлежности к классу
roc_score_0 = round(roc_auc_score(Y, [1/(1 + np.exp(- w_0[0]*X1[i] - w_0[1]*X2[i])) for i in range(len(X1))]), 3)
roc_score_10 = round(roc_auc_score(Y, [1/(1 + np.exp(- w_10[0]*X1[i] - w_10[1]*X2[i])) for i in range(len(X1))]), 3)
print(roc_score_0, roc_score_10)

0.927 0.936


In [11]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(C = 10, max_iter=1e9, random_state=42)

In [15]:
z = log_reg.fit(X, Y)

In [13]:
w1, w2 = z.coef_[0][0], z.coef_[0][1]
round(roc_auc_score(Y, [1/(1 + np.exp(- w1*X1[i] - w2*X2[i])) for i in range(len(X1))]), 6)

0.93181